In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import pandas as pd
import numpy as np
from scipy import stats
from scripts import *

# Overview

The purpose of this examination is to determine if there is a difference in successful observation based on satellite ownership.


$H_{0}$ = There is no difference in probability of successful observation.

$H_{A}$ = There is a difference in the probability of successful observation.

# Data Preparation

The data needs to be aggregated prior to processing. This will be achieved by:
1. Left join Satellites.csv and events.csv on SatNOGS satellite ID
2. Left join (joined satellites and events) and observations.csv on Observation ID

In [4]:
data = aggregate_data()
data['Ones'] = 1
data['Status_numeric'] = data['Status'].apply(lambda x: eval_obs_status(x))
data_groupby = data.groupby("countries").sum()
data_groupby['success_rate'] = data_groupby['Status_numeric'] / data_groupby['Ones']
data_groupby.head()

,Observation_id,Status_Message,norad_cat_id_x,transmitter,schema,observation_id,station_id,norad_cat_id_y,Ones,Status_numeric,success_rate
countries,,,,,,,,,,,
ALG,496324406,9700,4053533,0.0,0.0,4.963244e+08,175084.0,4053533.0,97,97,1.000000
ARGN,2079633192,42300,16489204,0.0,0.0,2.079633e+09,620965.0,16489204.0,423,423,1.000000
ASRA,1366257111,27300,11680032,0.0,0.0,1.366257e+09,541535.0,11680032.0,273,273,1.000000
AU,12585564,100,299851,0.0,0.0,1.258556e+07,1913.0,299851.0,3,2,0.666667
AUS,1221712233,12500,12384542,0.0,0.0,1.221712e+09,346571.0,12384542.0,273,199,0.728938


Next, drop the countries that have less than 30 observations

In [5]:
data_groupby_30plus = data_groupby[data_groupby['Ones'] >= 30]
print(f'Only {data_groupby_30plus.shape[0]} countries remain for hypothesis testing')

Only 55 countries remain for hypothesis testing


Next, define functions to save the hypothesis testing results and start testing

# Hypothesis Testing

In [6]:
testing_dir = "../hypothesis_tests/countries/"

In [7]:
indexes = data_groupby_30plus.index
hypothesis_test_countries(data, indexes, alpha = 0.05)

/home/ada/CodeProjects/SatNOGS2/Notebooks/scripts.py:48: RuntimeWarning: invalid value encountered in double_scalars
  equal_var = True if (((var1 / var2) <= 4) & ((var2 / var2) <=4) ) else False
/home/ada/CodeProjects/SatNOGS2/Notebooks/scripts.py:48: RuntimeWarning: divide by zero encountered in double_scalars
  equal_var = True if (((var1 / var2) <= 4) & ((var2 / var2) <=4) ) else False


In [8]:
hypothesis_test_greater_counts(data, indexes, alpha = 0.05)